# Anamoly Detection

In [38]:
import numpy as np
import pandas as pd
import sklearn
import scipy
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report,accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

In [39]:
data = pd.read_csv('Data from OBD\OBD_dataset_for_ml_1.csv',sep=',')
data.head()

,GPS Time,Device Time,Longitude,Latitude,GPS Speed (Meters/second),Horizontal Dilution of Precision,Altitude,Bearing,G(x),G(y),G(z),G(calibrated),Engine Coolant Temperature(°C),Engine RPM(rpm),Intake Air Temperature(°C),Engine Load(%),Mass Air Flow Rate(g/s),Throttle Position(Manifold)(%),Class
0,Thu Nov 02 15:44:24 CDT 2017,44:25.1,-106.000000,30.527469,26.490318,133,209,346.8,0.540592,7.191109,6.718781,-0.000509,85,1712.50,31,21.568628,7.590000,18.431374,1
1,Thu Nov 02 15:44:25 CDT 2017,44:26.1,-97.662546,30.482689,0.000000,101,225,0.0,0.275889,3.720857,8.191541,-0.063443,75,736.50,30,34.509804,2.750000,15.686275,0
2,Thu Nov 02 15:44:26 CDT 2017,44:27.1,-97.690020,30.527469,26.490318,133,209,346.8,0.936458,4.970899,7.908143,-0.024066,85,2238.25,31,96.078430,10.250000,22.352942,0
3,Thu Nov 02 15:44:27 CDT 2017,44:28.1,-97.756517,30.545218,26.360070,31,224,250.0,2.523680,6.888482,7.615170,0.096887,85,2219.50,31,67.450981,24.790001,30.588236,0
4,Thu Nov 02 15:44:27 CDT 2017,44:29.1,-97.770798,30.572977,13.070000,20,259,68.2,0.313047,7.536257,5.370520,-0.037131,89,1053.00,32,33.333332,3.480000,16.078432,0


In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1356 entries, 0 to 1355
Data columns (total 19 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   GPS Time                           1356 non-null   object 
 1    Device Time                       1356 non-null   object 
 2    Longitude                         1356 non-null   float64
 3    Latitude                          1356 non-null   float64
 4   GPS Speed (Meters/second)          1356 non-null   float64
 5    Horizontal Dilution of Precision  1356 non-null   int64  
 6    Altitude                          1356 non-null   int64  
 7    Bearing                           1356 non-null   float64
 8    G(x)                              1356 non-null   float64
 9    G(y)                              1356 non-null   float64
 10   G(z)                              1356 non-null   float64
 11   G(calibrated)                     1356 non-null   float

In [41]:
data.describe()

,Longitude,Latitude,GPS Speed (Meters/second),Horizontal Dilution of Precision,Altitude,Bearing,G(x),G(y),G(z),G(calibrated),Engine Coolant Temperature(°C),Engine RPM(rpm),Intake Air Temperature(°C),Engine Load(%),Mass Air Flow Rate(g/s),Throttle Position(Manifold)(%),Class
count,1356.000000,1356.000000,1356.000000,1356.000000,1356.000000,1356.000000,1356.000000,1356.000000,1356.000000,1356.000000,1356.000000,1356.000000,1356.000000,1356.000000,1356.000000,1356.000000,1356.000000
mean,-98.314269,30.536503,15.019647,52.460914,238.345133,211.945133,0.572515,6.510720,6.482762,0.016298,84.170354,1839.693584,31.573746,55.497427,14.550103,22.547574,0.193215
std,2.294911,0.030855,12.926928,47.350128,48.214010,140.549289,1.528175,2.076501,2.229453,0.071754,5.644947,987.867036,2.943957,26.489267,14.178106,8.887587,0.394966
min,-109.000000,30.482689,0.000000,11.000000,60.000000,0.000000,-7.692474,-2.073723,-4.262598,-0.305164,56.000000,614.000000,30.000000,16.470589,2.090000,14.901961,0.000000
25%,-97.767898,30.508071,2.926175,18.000000,218.000000,38.150000,-0.235302,5.532770,5.567860,-0.019881,85.000000,1074.562500,30.000000,32.156864,2.980000,15.686275,0.000000
50%,-97.709025,30.550567,14.485071,29.000000,230.000000,258.550000,0.600083,6.680282,6.784478,0.014268,86.000000,1835.375000,31.000000,48.235294,5.475000,16.862745,0.000000
75%,-97.687068,30.557920,23.985206,91.000000,253.000000,336.125000,1.576419,7.715458,7.848844,0.049367,86.000000,2390.125000,32.000000,81.568626,25.802501,27.450981,0.000000
max,-97.662474,30.574231,73.000000,246.000000,454.000000,359.900000,4.602767,11.798932,19.633602,0.806779,101.000000,7951.000000,55.000000,100.000000,74.000000,66.666664,1.000000


In [42]:
data.shape

(1356, 19)

## Exploratory Data Analysis

In [43]:
data.isnull().values.any()

False

In [44]:
## Get the anomaly and the normal dataset 

anamoly = data[data['Class']==1]

normal = data[data['Class']==0]

In [45]:
data1=data

In [46]:
#Determine the number of anomaly and valid transactions in the dataset

anomaly = data1[data1['Class']==1]

valid = data1[data1['Class']==0]

outlier_fraction = len(anomaly)/float(len(valid))

In [47]:
print(outlier_fraction)

print("Anomaly Cases : {}".format(len(anomaly)))

print("Valid Cases : {}".format(len(valid)))

0.23948811700182815
Anomaly Cases : 262
Valid Cases : 1094


# Check this 

In [48]:
#Create independent and Dependent Features
columns = data1.columns.tolist()
# Filter the columns to remove data we do not want 
#columns = [c for c in columns if c not in ["Class"]]
columns = [c for c in columns if c not in ["Class"] ]
# Store the variable we are predicting 
columns = columns[2:18]
target = "Class"

# Define a random state 
state = np.random.RandomState(42)

X = data1[columns]
Y = data1[target]
#X_outliers = state.uniform(low=0, high=1, size=(X.shape[0], X.shape[1]))
# Print the shapes of X & Y
#print(columns)
#print(X.shape)
#print(Y.shape)

print(columns)
columns[0]

[' Longitude', ' Latitude', 'GPS Speed (Meters/second)', ' Horizontal Dilution of Precision', ' Altitude', ' Bearing', ' G(x)', ' G(y)', ' G(z)', ' G(calibrated)', 'Engine Coolant Temperature(°C)', 'Engine RPM(rpm)', 'Intake Air Temperature(°C)', 'Engine Load(%)', 'Mass Air Flow Rate(g/s)', 'Throttle Position(Manifold)(%)']


' Longitude'

## Model Prediction


### Isolation Forest Algorithm :

### Local Outlier Factor(LOF) Algorithm


In [49]:
##Define the outlier detection methods

classifiers = {
    "Isolation Forest":IsolationForest(n_estimators=100, max_samples=len(X), 
                                       contamination=outlier_fraction,random_state=state, verbose=0),
    "Local Outlier Factor":LocalOutlierFactor(n_neighbors=20, algorithm='auto', 
                                              leaf_size=30, metric='minkowski',
                                              p=2, metric_params=None, contamination=outlier_fraction)
   
}

In [50]:
type(classifiers)

dict

In [54]:
#n_outliers = len(Fraud)

n_outliers = 100


y_final_loc = []
y_final_if = []

#for j in range(11,17):
#    abc = [columns[1],columns[2],columns[3],columns[4],columns[5],columns[7],columns[8],columns[9],columns[10],columns[j]]
#    X = data1[abc]
    
for i, (clf_name,clf) in enumerate(classifiers.items()):
    #Fit the data and tag outliers
    if clf_name == "Local Outlier Factor":
        y_pred = clf.fit_predict(X)
        scores_prediction = clf.negative_outlier_factor_
    else:    
        clf.fit(X)
        scores_prediction = clf.decision_function(X)
        y_pred = clf.predict(X)
    #Reshape the prediction values to 0 for Valid transactions , 1 for Fraud transactions
    y_pred[y_pred == 1] = 0
    y_pred[y_pred == -1] = 1

    if clf_name == "Local Outlier Factor":
        y_final_loc.append(y_pred)
    else:
        y_final_if.append(y_pred)
    
    #n_errors = (y_pred != Y).sum()
    
    # Run Classification Metrics
    print(clf_name)
    print("Accuracy Score :")
    print(accuracy_score(Y,y_pred))
    print("\n")

    
y_loc = np.array(y_final_loc)
y_if =  np.array(y_final_if)

Isolation Forest
Accuracy Score :
0.8399705014749262


Local Outlier Factor
Accuracy Score :
0.68952802359882




In [55]:
y_loc

array([[1, 0, 1, ..., 0, 0, 0]])

In [56]:
print(y_loc.shape)

y_loc_t = np.transpose(y_loc)

(1, 1356)


In [57]:
y_loc.shape[1]

1356

In [58]:
for i in range(0,y_loc.shape[1]):
    if y_loc_t[i] == 1:
        print(i)

0
2
5
17
23
29
30
31
34
49
57
61
63
67
71
76
78
79
80
93
97
104
107
110
119
124
125
126
131
134
136
144
150
152
154
157
168
170
172
176
178
185
190
193
195
199
200
204
216
217
224
227
230
231
232
235
240
242
246
247
253
254
256
268
269
270
273
283
285
291
305
309
313
316
322
325
333
343
345
363
365
367
369
372
378
379
384
385
386
388
398
402
407
411
414
415
416
418
423
431
432
434
440
442
444
447
452
453
454
463
466
472
474
475
486
491
496
499
503
504
506
511
521
522
523
528
530
532
533
536
540
544
550
551
554
563
566
573
575
577
578
581
586
595
596
603
610
613
625
626
627
638
653
661
666
669
672
678
680
683
685
687
688
693
694
701
706
709
710
713
718
720
724
726
727
733
736
741
742
743
751
755
758
761
764
766
768
769
773
775
780
781
785
786
787
788
807
814
819
822
823
826
830
839
841
852
860
861
870
874
875
888
893
895
896
903
908
914
916
929
930
938
945
950
957
959
964
969
971
973
978
983
985
989
994
996
1004
1008
1010
1011
1013
1018
1021
1036
1041
1042
1044
1045
1054
1055
1057
1062


In [59]:
y_if.shape
y_if_t = np.transpose(y_if)

In [60]:
for i in range(0,y_loc.shape[1]):
    if y_if_t[i] == 1:
        print(i)

0
5
6
15
17
18
23
26
32
34
39
49
51
56
57
63
67
69
72
78
79
84
87
97
99
102
104
105
106
107
110
115
119
122
124
126
128
131
136
139
143
144
147
153
157
162
169
170
172
180
189
190
193
197
200
201
202
204
218
223
224
226
227
229
231
232
234
235
243
245
247
258
259
264
271
272
276
282
285
288
297
298
299
305
312
313
328
329
333
335
341
342
359
363
366
368
370
376
378
384
385
386
388
403
410
413
415
416
418
419
420
423
426
431
432
436
444
446
447
451
459
466
472
474
482
490
491
496
497
498
504
514
521
528
532
533
535
539
540
548
552
555
556
566
571
573
577
578
589
591
595
603
605
607
609
610
615
617
618
619
622
623
624
625
627
634
638
642
655
661
666
667
675
688
694
699
701
709
718
720
724
727
739
741
743
745
761
764
772
773
775
776
778
780
782
789
791
796
805
807
811
813
814
819
824
837
839
847
851
856
866
870
874
878
891
897
898
900
903
906
907
910
911
914
916
920
939
945
947
948
950
959
969
985
986
988
989
990
994
998
1001
1008
1010
1015
1016
1019
1023
1028
1029
1032
1034
1036
1042
104